
# Imports

In [ ]:
import re
import random
import json
import numpy as np
from numpy import percentile,median
import pandas as pd
from itertools import accumulate, chain, repeat, tee
import itertools
import category_encoders as ce
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import statsmodels.api as sm
import sklearn
import sklearn.datasets
import sklearn.ensemble
from sklearn.model_selection import GridSearchCV,train_test_split,cross_val_score,ShuffleSplit
from sklearn.metrics import mean_squared_error,r2_score, make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import plot_partial_dependence
from sklearn.inspection import partial_dependence
from sklearn.linear_model import LinearRegression

# Functions

In [ ]:
# Output the mapping from categorical to numerical values for each feature in a dataset.
# Binary Encoding (Not Supervised Case)
def get_dict(encoded,original):
    yellow_pages={}
    for feature in cat_features:
        filter_col = [col for col in encoded if col.startswith(feature)]
        bits = encoded[filter_col].values
        names = original[feature].values
        yellow_pages[feature] ={name:bit for name,bit in zip(names,bits)}  
    return yellow_pages

# Output the mapping from categorical to numerical values for each feature in a dataset.
# Leave One Out Encoding(Supervised Case)
def dictionary(original,encoded):
    dict={}
    categorical = original.select_dtypes(include=['object']).columns.values
    for i in categorical:
        dict[i]={}
        a = list(zip(original[i].values,encoded[i].values))
        unique_pairs=[]
        keys=[]
        for x in a:
            if x not in unique_pairs:
                unique_pairs.append(x)
        for j in unique_pairs:
            keys.append(j[0])
        if len(keys) > len(set(keys)):
            print(" mapping is not 1 - 1 ")
            return None
        else:
            l = {}
            for j in unique_pairs:
                l[j[0]] = j[1]
        dict[i] = l
    return dict

# Split or just suffle, encode data and return dictionary of encoding
def prepare_data(dataset,target,features,split=True,supervised=True):
    dataset = dataset[dataset[target].notnull()]
    if supervised == True:
        if split==True:
            X = dataset[features]
            y = dataset[target]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
            encoder = ce.leave_one_out.LeaveOneOutEncoder()
            encoder.fit(X_train,y_train)
            X_train_encoded = encoder.transform(X_train)
            X_test_encoded = encoder.transform(X_test)
            dict_ = dictionary(pd.concat([X_train,X_test],  ignore_index=True),\
                               pd.concat([X_train_encoded,X_test_encoded], ignore_index=True))
            return X_train_encoded, X_test_encoded, y_train, y_test, dict_
        else:
            shuffled = dataset.sample(frac=1)
            X = shuffled[features]
            y = shuffled[target]
            encoder = ce.leave_one_out.LeaveOneOutEncoder()
            encoder.fit(X,y)
            X_encoded = encoder.transform(X)
            dict_ = dictionary(X,X_encoded)
            return X_encoded,y, dict_
    else:
        shuffled = dataset.sample(frac=1)
        X = shuffled[features]
        y = shuffled[target]
        encoder = ce.binary.BinaryEncoder()
        X_encoded = encoder.fit_transform(X)
        if split==False:
            return X_encoded,y, X 
        else: 
            X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2)
            return X_train, X_test, y_train, y_test

# Calculate 95% Confidence Intervals
def ci(scores):
    intervals = st.t.interval(alpha=0.95,
                              df=len(scores)-1,
                              loc=np.mean(scores),
                              scale=st.sem(scores)) 
    return intervals[0],median(scores),intervals[1]

# Relative Root Mean Square
def rrmse(actual,predicted):
    avrg = sum(actual)/len(actual)
    numerator = mean_squared_error(actual,predicted,squared=False)
    denominator = avrg
    return numerator/denominator

my_rrmse = make_scorer(rrmse, greater_is_better=True)

# Break a list xs into n equal sublists
def chunk(xs, n):
    assert n > 0
    L = len(xs)
    s, r = divmod(L, n)
    widths = chain(repeat(s+1, r), repeat(s, n-r))
    offsets = accumulate(chain((0,), widths))
    b, e = tee(offsets)
    next(e)
    return [xs[s] for s in map(slice, b, e)]

# Get n unique perturbations of a set s
def findsubsets(s, n): 
    return [list(i) for i in itertools.combinations(s, n)] 

# Strech a shorter list based on where a longer has repeating streaks
def stretch(longer,shorter):
    if len(longer) <= len(shorter):
        return shorter
    list_set = list(set(longer))
    res = shorter.copy()
    for i in sorted(list_set):
        counter = longer.count(i)
        if counter > 1:
            index = longer.index(i)
            for j in range(counter -1):
                res.insert(index,res[index])
    return res

# Grid search Cross Validation parameter tuning
def best_params(model,X,y,target_name,grid):
    search = GridSearchCV(model,grid,cv=5,n_jobs=8,verbose=10)
    # Fit the random search model
    search.fit(X,y)
    print(search.best_params_)
    results = {}
    results['model'] = type(model).__name__
    results['target'] = target
    results['params'] = search.best_params_
  # write results to file
    return results

def performance(y_test, preds,model,target):
    """
    Get predictions and true values and scatter plot them.
    Additionally plot a linear regression line on the points
    of the Scatter plot and get its equation y = a *x + b.
    The closer the line to the y = x line the better the prediction.
    Additionally break the preds and truth paired set into 4 equal
    chunks and plot the mean and variance in order to visualize
    any trend to the error of the prediction.
    """
    path = '/home/dimitris/Documents/papers/grain_of_salt/paper/images/performances/'
#     model_name = type(model).__name__
    model_name = 'Random Forest'
    rmse = mean_squared_error(preds,y_test)
    r2 = r2_score(y_test,preds)
    
    #    separate truth and preds in chunks
    y_test_sorted, preds_sorted = zip(*sorted(zip(y_test, preds)))
    zipped = list(zip(preds_sorted,y_test_sorted))
    lists = chunk(zipped,4)
    
    #    make linear regression line on points (pred,truth)
    preds_extended = sm.add_constant(preds)
    linear_regressor = sm.OLS(y_test,preds_extended).fit()
    #   alpha coefficient of linear regression
    a = round(linear_regressor.params[1],2)
    #   intercept of linear regression
    b = round(linear_regressor.params[0],2)
    #   get the sign of the intercept
    score = lambda i: ("+" if i > 0 else "") + str(i)
    Y = a*y_test +b
    #   calculate the limits of the graph, as the min and max of both preds and truths
    min_ = np.min(np.array(np.min(y_test),np.min(preds)))
    max_ = np.max(np.array(np.max(y_test),np.max(preds)))
    #   array of x's for identity line
    x = np.linspace(min_,max_, 1000)
    
    ############## PLOT ##############
    fig = plt.figure(figsize=(7.5,7.5))
    # H0: The slope of the X variable is zero
    label = 'regresssion:'+'y='+str(a)+'x'+score(b) +\
    '\np-value: ' + str(np.format_float_scientific(linear_regressor.pvalues[1],precision=2))\
    +'\n'+'R^2: '+str(round(r2,2))
    #     +'\nRMSE '+str(round(rmse,2))+\
    plt.plot(y_test,Y,'green',label=label)
    # scatter plot of pred and truths
    plt.scatter(y_test, preds, alpha=0.5,label='preds')
    plt.xlim(min_,max_)
    plt.ylim(min_,max_)
    plt.xlabel('true values',fontsize = 18)
    plt.ylabel('predited',fontsize=18)
    title1 = '%s, %s' % (model_name,target)
    plt.title(title1,fontsize = 20)
    plt.legend(fontsize=16)
    plt.savefig(path+target+'.png')
    plt.show()
    plt.close()
    fig = plt.figure(figsize=(20,4))
    ax=[None]*4
    means=[]
    stds_=[]
    for list_,index in zip(lists,range(4)):
        res = [pred - truth for pred, truth in list_ ]
        mean = np.mean(res)
        std = np.std(res)
        means.append(mean)
        stds_.append(std)
        ax[index] = fig.add_subplot(1,4,index+1)
        ax[index].hist(res,bins=50,label='mean: '+str(round(mean,6)) + '\n' + 'st.deviation: '+str(round(std,6)))
        ax[index].set_title('slice: '+str(index +1))
        ax[index].legend()
    plt.show()
    plt.close()
    fig = plt.figure(figsize=(7.5,7.5))
    title2 = 'predicted - true: means & st.deviation, %s' % (target)
    plt.plot(means, '--ro',label='mean',linewidth=2)
    plt.plot(stds_ , '--go',label='st.deviation',linewidth=2)
    plt.title(title2,fontsize=20)
    plt.xlabel('groups',fontsize=18)
    plt.xticks([0,1,2,3],['1st','2nd','3rd','4th'])
    plt.legend(fontsize=16)
    plt.savefig(path+target+'_means.png')
    plt.show()
    plt.close()
    return None

# Data

In [ ]:
excel = pd.read_excel('./data/PIDE3.2+mcds.xlsx',convert_float=False)
excel.rename(columns = {
                        'ai_paper':'alpha',
                        'bi_paper':'beta',
                        'All Clusters/(Gbp*Gy)_hy':'All_clusters_hy',
                        'AllClusters/(Gbp*Gy)_ox':'All_clusters_ox',
                        'DSB/(Gbp*Gy)_hy':'DSBs_hy',
                        'DSB/(Gbp*Gy)_ox':'DSBs_ox'
                       }, inplace = True)
# 
targets = ['alpha','beta','DSBs_hy','DSBs_ox','All_clusters_hy','All_clusters_ox']
features = ['Cells','CellClass','CellOrigin','CellCycle','DNAcontent','Ion','Charge','IrradiationConditions','LET','Energy']
cat_features = ['Cells','CellCycle','Ion','CellClass','CellOrigin','IrradiationConditions']
excel.sample(frac=1).head(10)

# Hyperopt

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials,anneal, rand, mix,partial
import json
import os

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

rf_grid = {
    'max_depth': range(1,40),
    'n_estimators': range(2000),
    'min_samples_split': [2,3,4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_leaf': [1,2,3],
    'oob_score':[True,False],
    'max_samples':[0.8,0.9],
    'criterion':['mae','mse'],
    }

rf_space = {
#     'max_depth': hp.choice('max_depth', range(1,40)),
    'n_estimators': hp.choice('n_estimators', range(10,2000,20)),
    'min_samples_split':hp.choice('min_samples_split', [2,3,4]),
    'max_features': hp.choice('max_features',['auto', 'sqrt', 'log2']),
    'min_samples_leaf': hp.choice('min_samples_leaf',[1,2,3]),
    'oob_score':hp.choice('oob_score',[True,False]),
    'max_samples':hp.choice('max_samples',[0.8,0.9]),
    'criterion':hp.choice('criterion',['mae','mse'])
}

def hyperparameter_tuning(space):
    model=RandomForestRegressor(n_jobs=-1,
#                                 max_depth = int(space['max_depth']),
                                n_estimators =space['n_estimators'],
                                max_features = space['max_features'],
                                min_samples_split = int(space['min_samples_split']),
                                min_samples_leaf = int(space['min_samples_leaf']),
                                criterion = space['criterion'],
                                oob_score = space['oob_score'],
                                max_samples = space['max_samples'],
                               )

    model.fit(X,y)
    error = cross_val_score(model, X, y,n_jobs=-1,cv=5,scoring=my_rrmse).mean()
    print ("SCORE:", error)
    return {'loss':error, 'status': STATUS_OK, 'model': model}

best_parameters = []
for target in targets:
    X, y, original = prepare_data(excel,target,features,split=False,supervised=False)
    trials = Trials()
    mix_algo = partial(mix.suggest,
                       p_suggest=[(0.05, rand.suggest),
                                  (0.75, tpe.suggest),
                                  (0.20, anneal.suggest)])
    best = fmin(fn=hyperparameter_tuning,
                space=rf_space,
                algo=mix_algo,
                max_evals=150,
                trials=trials)
    print(best)
    for key in rf_grid:
        if type(rf_grid[key]) is list:
            try:
                best[key] = rf_grid[key][best[key]]
            except:
                print('MISSILES LAUNCHED!!!')
                print(rf_grid[key],best[key])
    best['n_jobs']=-1
    print(best)
    results = {}
    results['model'] = 'Random Forest'
    results['target'] = target
    results['params'] = best
    best_parameters.append(results)

with open('best_params.txt', 'w') as param_file:
    for element in best_parameters:
        param_file.write(json.dumps(element,cls=NpEncoder))
        param_file.write(os.linesep)

# Performance

In [ ]:
parameters = []
with open('best_params.txt') as f:
    for jsonObj in f:
        paramDict = json.loads(jsonObj)
        parameters.append(paramDict)

for obj in parameters:
    target = obj['target']
    X_train, X_test, y_train, y_test = prepare_data(excel,target,features,split=True,supervised=False)
    model = RandomForestRegressor()
    params = obj['params']
    model.set_params(**params)
    model.fit(X_train,y_train)
    preds = model.predict(X_test)
    test = np.asarray(y_test)
    performance(test,preds, model, target)

# Bootsrtap Confidence Intervals

In [ ]:
from sklearn.utils import resample
from sklearn.metrics import explained_variance_score,r2_score
import scipy.stats as st
from sklearn.preprocessing import OneHotEncoder
import sys
for obj in parameters:
    target = obj['target']
    X,y,original = prepare_data(excel,target,features,split=False,supervised=False)
    print('target: ',target,'\tsamples: ',len(X))
    # configure bootstrap
    n_iterations = 10
    values = X.join(y).values
    n_size = int(len(values) * 0.5)
    # run bootstrap
    stats = []
    for i in range(n_iterations):
        # prepare train and test sets
        train = resample(values, n_samples=n_size)
        test = np.array([x for x in values if x.tolist() not in train.tolist()])
        # name them more sensibly
        X_train = train[:,:-1]
        y_train = train[:,-1]
        X_test = test[:,:-1]
        y_test = test[:,-1]

        # fit model
        model = RandomForestRegressor()
        params = obj['params']
        model.set_params(**params)
        model.fit(X_train, y_train)

        # evaluate model
        predictions = model.predict(X_test)
        score = rrmse(y_test, predictions)
        print(score)
        stats.append(score)
    # confidence intervals
    print(ci(stats),'\t', target)


# Statistical Significance

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy.stats import ttest_rel
def statistical_model(X,y,target,features):
    dataset = pd.concat([X,y],axis=1)
    formula = target + ' ~ '+' + '.join(features)
    model = smf.glm(formula = formula,data=dataset, family=sm.families.Poisson()).fit()
    model = smf.ols(formula ,data=dataset).fit()
    return model
    
families =[
    sm.families.Gamma(),
    sm.families.Binomial(),
    sm.families.Gaussian(),
    sm.families.Poisson(),
    
]
for obj in parameters:
    target = obj['target']
    print(target)
    stat_mse=[]
    rf_mse = []
    for i in range(10):
        dataset = excel[excel[target].notnull()]
        X = dataset[features]
        y = dataset[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        
        encoder = ce.leave_one_out.LeaveOneOutEncoder()
        encoder.fit(X_train,y_train)
        X_train_encoded = encoder.transform(X_train)
        X_test_encoded = encoder.transform(X_test)
        
        model = RandomForestRegressor()
        params = obj['params']
        model.set_params(**params)
        model.fit(X_train_encoded,y_train)
        rf_preds = model.predict(X_test_encoded)
        rf_mse.append(rrmse(y_test,rf_preds))

        stat_model = statistical_model(X_train_encoded,y_train,target,features)
        stat_preds = stat_model.predict(X_test_encoded)
        stat_mse.append(rrmse(y_test,stat_preds))
        
    stat, p = ttest_rel(stat_mse, rf_mse)
    print('\tMean rf_mse=%.3f,\n\tMean stat_mse=%.3f,\n\tStatistics=%.3f,\n\tp=%s' \
          % (sum(rf_mse)/len(rf_mse),sum(stat_mse)/len(stat_mse),stat, np.format_float_scientific(p,precision=3)))
    # interpret
    alpha = 0.05
    if p > alpha:
        print('\tSame distributions (fail to reject H0)')
    else:
        print('\tDifferent distributions (reject H0)')


# Interpretations

In [ ]:
from sklearn.linear_model import LinearRegression
import lime
import lime.lime_tabular
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.inspection import partial_dependence
from sklearn.inspection import plot_partial_dependence
import sys

path = '/home/dimitris/Documents/papers/grain_of_salt/paper/images/'

def plot_importance(model,features, target,X):
    file = '%simportances/%s.png' % (path,target)
    title = 'Feature Importance, %s' % (target)
    ########## Importance ###############
    short_features = ['Ir_cond' if a == 'IrradiationConditions' else a for a in features]
    unpacked_importance = model.feature_importances_
    aggregate_importnance = []
    for feature in features:
        filter_col = [col for col in X if col.startswith(feature)]
        indeces = [X.columns.get_loc(col) for col in filter_col]
        aggregate_importnance.append(sum([unpacked_importance[index] for index in indeces]))
    x = aggregate_importnance
    max_ = max(x)*1.1
    y = short_features
    sorted_list = sorted(list(zip(y,x)),key=lambda tup: tup[1])
    df = pd.DataFrame(sorted_list, columns = ['Features', 'Importance'])
    plt.figure(figsize=(8, 8))
    splot=sns.barplot(x='Features',y='Importance',data=df)
    
    for p,name in zip(splot.patches,df['Features'].values):
        offset = len(name)*6
        if len(name)/40+p.get_height() > max_ :
            print(name)
            offset = -len(name)*5
        splot.annotate(name, 
        (p.get_x() + p.get_width() / 2., p.get_height()), 
        ha = 'center', va = 'center', 
        size=16,
        xytext = (0, offset), 
        textcoords = 'offset points',
        rotation=90)
    plt.tick_params(labelbottom=False,
                    axis='x',
                    which='both',
                    bottom=False,
                    top=False)
    
    plt.xlabel("Features", size=16)
    plt.ylabel("Importance", size=16)
    plt.title(title,fontsize=18)
    plt.savefig(file)
    plt.show()
    plt.close()
    return None

# Plot partial dependence for categorical features
def categorical_pd(model,X,feature,dict_):
    if feature != 'Cells':
        return None
    file = '%spd/%s_%s.png' % (path,target,feature)
    filter_col = [col for col in X if col.startswith(feature)]
    yellow_pages = dict_[feature]
    pd = []
    for name in yellow_pages:
        X_copy = X.copy()
        for col,value in zip(filter_col,yellow_pages[name]):
            X_copy[col].values[:] = value
        preds = model.predict(X_copy)
        pd.append(np.mean(preds))
    x,y = yellow_pages.keys() ,pd
    title= 'Partial Dependence, %s' % (feature)
    if feature in ['Cells','Ion']:
        size = (16, 7.5)
    else:
        size = (7.5,7.5)
    rotation = (0,90)[feature =='Cells']
    zipped = list(zip(x,y))
    sorted_zipped = sorted(zipped, key=lambda tup: tup[1])
    locs = list(range(len(x)))
    x,y=zip(*sorted_zipped)
    width = (0.8,0.2)[len(locs) == 2]
    plt.figure(figsize=size)
    plt.title(title, fontsize =20)        
    plt.xticks(locs,x,rotation=rotation,fontsize=12)
    plt.xlabel(feature,fontsize=16)
    plt.ylabel('Predicted Outcome',fontsize=16)
    plt.bar(x,y)
    plt.ylim([min(y)*0.9,max(y)*1.05])
    plt.savefig(file)
    plt.show()
    plt.close()
    return None

# Plot partial dependence for numerical features
def numerical_pd(feature,model,X):
    file = '%spd/%s_%s.png' % (path,target,feature)
    y,x = partial_dependence(model, X, [feature])
    x_,y_ = x[0].tolist(),y[0].tolist()
    title= 'Partial Dependence, %s' % (feature)
    ymax = max(y_)
    xpos = y_.index(ymax)
    xmax = x_[xpos]
    label = feature + '= ' + str(round(xmax,2))
    plt.figure(figsize=(8,8))
    plt.title(title, fontsize =20)
    plt.xlabel(feature,fontsize=16)
    plt.ylabel('Predicted Outcome',fontsize=16)
    plt.plot(x_,y_,'tab:red',linewidth=2)
    plt.annotate(label, xy=(xmax, ymax), fontsize=16)
    plt.savefig(file)
    plt.show()
    plt.close()
    return None

def two_way(est,X_train,target):
    pair = ('Energy', 'LET')
    file = '%spd/%s_%s_%s.png' % (path,target,pair[0],pair[1])
    title = target+'\nEnergy - LET Partial Dependence'
    fig = plt.figure()
    pdp, axes = partial_dependence(est, X_train, features=pair,
                                   grid_resolution=20)
    XX, YY = np.meshgrid(axes[0], axes[1])
    Z = pdp[0].T
    ax = Axes3D(fig)
    surf = ax.plot_surface(XX, YY, Z, 
                           cmap=plt.cm.viridis
                           , edgecolor='k')
    ax.set_xlabel(pair[0],fontsize=14)
    ax.set_ylabel(pair[1],fontsize=14,rotation=15)
    ax.set_zlabel('Partial dependence')
    #  pretty init view
    ax.view_init(elev=22, azim=30)
    plt.colorbar(surf)
    plt.yticks(rotation=15)
    plt.suptitle(title,fontsize=18)
    plt.subplots_adjust(top=0.9)
    plt.savefig(file)
    plt.show()
    plt.close()
    return None
        
def local_interpretation(model,X,instance, target, original_instance):
    file = '%slime/%s.png' % (path,target)
    title = 'Local interpretation: %s' % (target)
    predict = lambda z: model.predict(z).astype(float)
    prediction = predict(instance.reshape(1,-1))
    explainer = lime.lime_tabular.LimeTabularExplainer(X.values,
                                                       feature_names=X.columns.values,
                                                       verbose=False,
                                                       mode='regression')
    
    explanation = explainer.explain_instance(instance,
                                             predict,
                                             num_features=28,
                                             num_samples=100,
                                             model_regressor=LinearRegression(fit_intercept=False)   
                                            )

    unpack = lambda pair:(pair[0].split()[0],float(pair[0].split()[2])) if (len(pair[0].split()) == 3)\
             else (pair[0].split()[2], float(pair[0].split()[4]))
                            
    lime_list = explanation.as_list()
    results = []
    for feature in cat_features:
        impact = 0
        filter_col = [col for col in X if col.startswith(feature)]
        for item in lime_list:
            unpacked_item = unpack(item)
            if unpacked_item[0] in filter_col:
                impact += item[1]
        name = original_instance[feature]
        result = '%s=%s'% (feature,name)
        results.append((result,impact))
    encoded_features = [col for col in X for feature in cat_features if col.startswith(feature)]
    unpacked_list = list(map(unpack,lime_list))
    for lime_item, unpacked_item in zip(lime_list,unpacked_list):
            if unpacked_item[0] not in encoded_features:
                results.append(lime_item)
    results = sorted(results ,key = lambda tup:abs(tup[1]))

    impacts = [i[1] for i in results]
    y_labels = [i[0] for i in results]
    impact_series = pd.Series(impacts)
    # Plot the figure.
    fig, ax = plt.subplots(figsize=(8,8))
    ax.barh(list(range(len(impacts))),impacts)
    ax.set_title('Local Interpretation, '+target, fontsize=20)
    ax.set_ylabel('Impacts', fontsize=16)
    ax.set_xlabel('Predicted Outcome', fontsize=16)
    max_value = max(impacts)
    min_value = min(impacts)
    unit = (abs(1.4*min_value)+max_value*1.1)/500
    ax.set_xlim([1.4*min_value, max_value*1.1])
    ax.set_yticklabels([])
    rects = ax.patches
    # Annotations
    for rect,impact,y_label in zip(rects,impacts,y_labels):
        rect.set_height(0.66)
        y_value = rect.get_y() + rect.get_height()/2
        # Number of points between bar and label. Change to your liking.
        space = 2 if impact > 0 else -2
        color = 'g' if impact >0 else 'r'
        rect.set_color(color)
        ha = 'left' if impact > 0 else 'right'
        offset = unit*2 if impact > 0 else -unit*2
        label = "{:.1f}".format(impact)
        ax.text(impact+offset,y_value,label,size=12,transform=ax.transData,ha=ha,va='center')
        ax.text(-offset,y_value,
                y_label,size=12,
                transform=ax.transData,
                ha='left' if ha=='right' else 'right',
                va='center')
    # prediction
    ax.text(0.985, 0.015, 'prediction: '+str(round(prediction[0],2)),size=12,
            ha="right", va="bottom",
            transform=ax.transAxes, 
            bbox=dict(boxstyle="square",
#             ec='tab:gray',
            fc='white',
            ))
    plt.savefig(file)                   
    plt.show()
    plt.close()
    return result

parameters = []
with open('best_params.txt') as f:
    for jsonObj in f:
        paramDict = json.loads(jsonObj)
        parameters.append(paramDict)
        
for obj in parameters:
    target = obj['target']
    X , y, original = prepare_data(excel,target,features,split=False,supervised=False)
    print(target)
    dict_ = get_dict(X,original)
    model = RandomForestRegressor()
    params = obj['params']
    model.set_params(**params)
    model.fit(X,y)
    for feature in features:
        if feature in cat_features:
            categorical_pd(model,X,feature,dict_)
        else:
            numerical_pd(feature,model,X)
    plot_importance(model,features, target,X)
    two_way(model,X,target)
    local_interpretation(model,X,X.iloc[10].values,target,original.iloc[10])
